In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Source files
brands=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\brands.csv',index_col='brand_id')
categories=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\categories.csv',index_col='category_id')
customers=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\customers.csv',index_col='customer_id')
orders=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\orders.csv',index_col='order_id')
order_items=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\order_items.csv')
products=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\products.csv',index_col='product_id')
staffs=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\staffs.csv',index_col='staff_id')
stores=pd.read_csv(r'C:\Users\hello\OneDrive\ME\ME\LEARNING\Python\Projects\Bikers club\Bikers\stores.csv',index_col='store_id')

# CONVERTING Object data type into datetime64 (for time series Analysis)
orders['order_date']=pd.to_datetime(orders['order_date'])
orders['shipped_date']=pd.to_datetime(orders['shipped_date'])
orders['required_date']=pd.to_datetime(orders['required_date'])

# editing in staff table
staffs=staffs.drop('active',axis=1)
staffs=staffs.drop('manager_id',axis=1)

# adding total sale value-revenue
revenue_formula=(order_items['quantity']*order_items['list_price'])-(order_items['discount']*order_items['quantity'])
order_items=order_items.assign(revenue=revenue_formula)

In [3]:
## PARAMETER 1                                                          
# Date based Analytics

# Function to merge all required tables
def merge_all():
    Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                     .merge(customers, how='inner', on='customer_id')
                     .merge(stores, how='inner', on='store_id')
                     .merge(products, how='inner', on='product_id')
                     .merge(brands, how='inner', on='brand_id')
                     .merge(categories,how='inner',on='category_id'))

    Result=Merge_all
    return Result

# creating merged dataframe
Final=merge_all()

# Filtering dataframe by date inputs
print("Dataset covers the period from January 1, 2016 to December 31, 2018.")
start_date=input('Starting Date (YYYY-MM-DD): ')
end_date=input('Ending Date (YYYY-MM-DD): ')

try:
    if start_date > end_date:
        print('End date cannot be smaller than the start date,please try again')
    else:
            Required=Final[Final['order_date'].between(start_date,end_date)]
            
            # Required answers-1 (Revenue, Total orders, AOV) (set of 3 major metrics)
            revenue=Required['revenue'].sum().round(2)
            total_orders=Required['order_id'].nunique()
            AOV=(revenue/total_orders).round(2)
            
            # Required answers-2 (Day with the highest revenue and the greatest number of orders placed)
            R2=Required.loc[:,['order_date','order_id','revenue']]\
                       .groupby(['order_date']).agg(total_revenue=('revenue','sum'),
                                                    total_orders=('order_id','nunique'))
            
            highest_revenue_day=R2['total_revenue'].sort_values(ascending=False).head(1)
            highest_order_day=R2['total_orders'].sort_values(ascending=False).head(1)
            
            # Required answers-3 (Biggest customer according to revenue)
            R3=Required.groupby(['customer_id','first_name','last_name'],as_index=False)\
                       .agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            
            # Required answers-4,5,6 (Highest grossing product, brand and category)
            R4=Required.groupby(['product_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R5=Required.groupby(['brand_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R6=Required.groupby(['category_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            
            # Printing required answers
            print('') # Required answer 1
            print('Total revenue generated in this period: ',revenue)
            print('Total orders placed: ',total_orders)
            print('Avergae Order Value: ',AOV)
            print('') # Required answer 2
            print(f'Highest revenue day is: {highest_revenue_day.index[0].date()} with total revenue of: {highest_revenue_day.values[0].round(2)}')
            print(f'Day with most orders placed is: {highest_order_day.index[0].date()} with: {highest_order_day.values[0]}')
            print('') # Required answer 3
            print(f'Biggest customer was: {R3['first_name'].iloc[0]} {R3['last_name'].iloc[0]} (customer id: {R3['customer_id'].iloc[0]}) with revenue of: {R3['total_revenue'].iloc[0].round(2)} ' )
            print('') # Required answer 4,5,6
            print(f'Highest selling product is : {R4.index[0]} with total sales of: {R4['total_revenue'].iloc[0].round(2)}')
            print(f'Highest selling brand is : {R5.index[0]} with total sales of: {R5['total_revenue'].iloc[0].round(2)}')
            print(f'Highest selling category is : {R6.index[0]} with total sales of: {R6['total_revenue'].iloc[0].round(2)}')
            
            Final_df=Required.loc[:,['order_date','order_id','customer_id','first_name','last_name','product_name','brand_name','category_name']]
            display(Final_df)
except Exception:
    print("Invalid input. Make sure dates are in the correct format (YYYY-MM-DD) and are valid calendar dates.")

Dataset covers the period from January 1, 2016 to December 31, 2018.


Starting Date (YYYY-MM-DD):  2016-12-28
Ending Date (YYYY-MM-DD):  2018-09-12



Total revenue generated in this period:  5861168.8
Total orders placed:  979
Avergae Order Value:  5986.89

Highest revenue day is: 2018-04-17 with total revenue of: 68269.63
Day with most orders placed is: 2018-04-12 with: 9

Biggest customer was: Pamelia Newman (customer id: 10) with revenue of: 37799.6 

Highest selling product is : Trek Domane SLR 6 Disc - 2017 with total sales of: 236495.04
Highest selling brand is : Trek with total sales of: 3834336.79
Highest selling category is : Road Bikes with total sales of: 1840397.78


,order_date,order_id,customer_id,first_name,last_name,product_name,brand_name,category_name
1766,2016-12-28,633,1040,Shantae,Hammond,Electra Townie Original 21D - 2016,Electra,Cruisers Bicycles
1767,2016-12-28,633,1040,Shantae,Hammond,Electra Cruiser 1 (24-Inch) - 2016,Electra,Cruisers Bicycles
1768,2016-12-28,633,1040,Shantae,Hammond,Heller Shagamaw Frame - 2016,Heller,Mountain Bikes
1769,2016-12-28,633,1040,Shantae,Hammond,Surly Straggler - 2016,Surly,Cyclocross Bicycles
1770,2016-12-29,634,27,Santos,Valencia,Trek Fuel EX 8 29 - 2016,Trek,Mountain Bikes
...,...,...,...,...,...,...,...,...
4704,2018-08-25,1610,15,Linnie,Branch,Surly Troll Frameset - 2017,Surly,Mountain Bikes
4705,2018-08-25,1610,15,Linnie,Branch,Trek Domane ALR Frameset - 2018,Trek,Road Bikes
4706,2018-09-06,1611,6,Lyndsey,Bean,Electra Loft Go! 8i - 2018,Electra,Electric Bikes
4707,2018-09-06,1611,6,Lyndsey,Bean,Trek Domane SL 6 - 2018,Trek,Road Bikes


In [4]:
## PARAMETER 2
# Customer Analytics

# Function to merge all required tables
def merge_all():
    Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                     .merge(customers, how='inner', on='customer_id')
                     .merge(stores, how='inner', on='store_id')
                     .merge(products, how='inner', on='product_id')
                     .merge(brands, how='inner', on='brand_id')
                     .merge(categories,how='inner',on='category_id'))

    Result=Merge_all
    return Result

# creating merged dataframe
Final=merge_all()

try:
# customer id input
    customer_id_input=int(input('Please type the required customer id: '))
    print('')
    unique_customers=Final['customer_id'].unique()

    if customer_id_input in unique_customers:
        Required=Final[Final['customer_id']==customer_id_input]

        # customer info
        customer_name=Required['first_name'].iloc[0]+' '+Required['last_name'].iloc[0]

        # Required answers-1 (Revenue, Total orders, AOV) (set of 3 major metrics)
        Revenue=Required['revenue'].sum().round(2)
        Total_orders=Required['order_id'].nunique()
        AOV=(Revenue/Total_orders).round(2)

        # Required answers 2,3,4
        R2=Required.groupby(['product_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        R3=Required.groupby(['brand_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        R4=Required.groupby(['category_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        R5=Required.groupby(['store_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)

        
        # Printing required answer
        print('Customer id: ',customer_id_input)
        print('Name: ',customer_name)
        print('Revenue generated: ',Revenue)
        print('Total orders: ',Total_orders)    
        print('Average Order Value: ',AOV)
        print('')
        print(f'Top grossing product: {R2.index[0]} with revenue of: {R2['total_revenue'].iloc[0].round(2)} ')
        print(f'Top grossing brand: {R3.index[0]} with revenue of: {R3['total_revenue'].iloc[0].round(2)} ')
        print(f'Top grossing category: {R4.index[0]} with revenue of: {R4['total_revenue'].iloc[0].round(2)} ')
        print(f'Top grossing store: {R5.index[0]} with revenue of: {R5['total_revenue'].iloc[0].round(2)} ')

        Final_df=Required.loc[:,['order_date','order_id','revenue','product_name','brand_name','category_name']]
        display(Final_df)
    else:
        print('This customer id does not exist in our database, please try again')
except Exception:
    print("Invalid input. Make sure input are in correct numerical (integer) format ")

Please type the required customer id:  7



Customer id:  7
Name:  Latasha Hays
Revenue generated:  7707.47
Total orders:  3
Average Order Value:  2569.16

Top grossing product: Trek Crockett 7 Disc - 2018 with revenue of: 5999.84 
Top grossing brand: Trek with revenue of: 6209.73 
Top grossing category: Cyclocross Bicycles with revenue of: 5999.84 
Top grossing store: Baldwin Bikes with revenue of: 7707.47 


,order_date,order_id,revenue,product_name,brand_name,category_name
308,2016-03-03,104,897.80,Pure Cycles William 3-Speed - 2016,Pure Cycles,Cruisers Bicycles
309,2016-03-03,104,599.94,Electra Townie Original 7D EQ - 2016,Electra,Comfort Bicycles
4468,2018-04-13,1532,5999.84,Trek Crockett 7 Disc - 2018,Trek,Cyclocross Bicycles
4690,2018-06-17,1604,209.89,Trek Precaliber 16 Girl's - 2018,Trek,Children Bicycles


In [5]:
# PARAMETER 3
# Stores Analytics

# Function to merge all required tables
def merge_all():
    Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                     .merge(customers, how='inner', on='customer_id')
                     .merge(stores, how='inner', on='store_id')
                     .merge(products, how='inner', on='product_id')
                     .merge(brands, how='inner', on='brand_id')
                     .merge(categories,how='inner',on='category_id'))

    Result=Merge_all
    return Result

# creating merged dataframe
Final=merge_all()

try:
# store id input
    store_id_input=int(input('Please type store id (1-3): '))
    print('')
    unique_store_ids=Final['store_id'].unique()

    if store_id_input in unique_store_ids:
        Required=Final[Final['store_id']==store_id_input]

        # store info
        store_name=Required['store_name'].iloc[0]

        # Required answers-1 (Revenue, Total orders, AOV) (set of 3 major metrics)
        Revenue=Required['revenue'].sum().round(2)
        Total_orders=Required['order_id'].nunique()
        AOV=(Revenue/Total_orders).round(2)

        # Required answers 2,3,4,5
        R2=Required.groupby(['customer_id','first_name','last_name'],as_index=False)\
                   .agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        
        R3=Required.groupby(['product_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        R4=Required.groupby(['brand_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
        R5=Required.groupby(['category_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)

        
        # Printing required answer
        print('Store id: ',store_id_input)
        print('Name: ',store_name)
        print('Revenue generated: ',Revenue)
        print('Total orders: ',Total_orders)    
        print('Average Order Value: ',AOV)
        print('')
        print(f'Biggest customer : {R2['first_name'].iloc[0]} {R2['last_name'].iloc[0]} (customer id: {R2['customer_id'].iloc[0]}) with revenue of: {R2['total_revenue'].iloc[0].round(2)} ' )
        print(f'Top grossing product: {R3.index[0]} with revenue of: {R3['total_revenue'].iloc[0].round(2)} ')
        print(f'Top grossing brand: {R4.index[0]} with revenue of: {R4['total_revenue'].iloc[0].round(2)} ')
        print(f'Top grossing category: {R5.index[0]} with revenue of: {R5['total_revenue'].iloc[0].round(2)} ')

        Final_df=Required.loc[:,['order_date','order_id','revenue','product_name','brand_name','category_name']]
        display(Final_df)
    else:
        print('This store id does not exist in our database, pick from 1-3')
except Exception:
    print("Invalid input. Make sure input are in correct numerical (integer) format.")

Please type store id (1-3):  3



Store id:  3
Name:  Rowlett Bikes
Revenue generated:  962521.97
Total orders:  174
Average Order Value:  5531.74

Biggest customer : Elinore Aguilar (customer id: 61) with revenue of: 29659.52 
Top grossing product: Trek Slash 8 27.5 - 2016 with revenue of: 83997.97 
Top grossing brand: Trek with revenue of: 612788.07 
Top grossing category: Mountain Bikes with revenue of: 348572.34 


,order_date,order_id,revenue,product_name,brand_name,category_name
93,2016-01-20,31,3361.88,Surly Straggler 650b - 2016,Surly,Cyclocross Bicycles
94,2016-01-20,31,5999.58,Trek Conduit+ - 2016,Trek,Electric Bikes
144,2016-01-31,50,999.94,Surly Wednesday Frameset - 2016,Surly,Mountain Bikes
145,2016-01-31,50,1199.84,Electra Townie Original 7D EQ - 2016,Electra,Comfort Bicycles
146,2016-01-31,50,539.58,Electra Cruiser 1 (24-Inch) - 2016,Electra,Cruisers Bicycles
...,...,...,...,...,...,...
4717,2018-11-28,1614,4599.84,Trek Emonda ALR 6 - 2018,Trek,Road Bikes
4718,2018-11-28,1614,539.58,Electra Cruiser 1 - 2016/2017/2018,Electra,Cruisers Bicycles
4719,2018-12-28,1615,4599.58,Trek Verve+ Lowstep - 2018,Trek,Electric Bikes
4720,2018-12-28,1615,899.92,Electra Tiger Shark 3i - 2018,Electra,Cruisers Bicycles


In [12]:
# PARAMETER 4
# Product/ Brand/ Category Analytics

def product_analysis():
    def merge_all():
        Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                         .merge(customers, how='inner', on='customer_id')
                         .merge(stores, how='inner', on='store_id')
                         .merge(products, how='inner', on='product_id')
                         .merge(brands, how='inner', on='brand_id')
                         .merge(categories,how='inner',on='category_id'))
    
        Result=Merge_all
        return Result
    
    # creating merged dataframe
    Final=merge_all()
    
    try:
        product_id_input=int(input('Please type the required product id here (1-315) : '))
        print('')
        unique_ids=Final['product_id'].unique()
        
        if product_id_input in unique_ids:
            Required=Final[Final['product_id']==product_id_input]
    
            # Product informaton
            product_name=Required['product_name'].iloc[0]
            product_brand=Required['brand_name'].iloc[0]
            product_category=Required['category_name'].iloc[0]
    
            # Required answers-1 (Revenue, Total orders, AOV) (set of 3 major metrics)
            Revenue=Required['revenue'].sum().round(2)
            Total_orders=Required['order_id'].nunique()
            AOV=(Revenue/Total_orders).round(2)
    
            # Required answers 2,3,4
            R2=Required.groupby(['customer_id','first_name','last_name'],as_index=False)\
                       .agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R3=Required.groupby(['store_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R4=Required.groupby(['order_date']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
    
            # Printing required answer
            print('Product id: ',product_id_input)
            print('Name: ',product_name)
            print('Brand: ',product_brand)
            print('Category: ',product_category)
            print('')
            print('Revenue generated: ',Revenue)
            print('Total orders: ',Total_orders)    
            print('Average Order Value: ',AOV)
            print('')
            print(f'Biggest customer : {R2['first_name'].iloc[0]} {R2['last_name'].iloc[0]} (customer id: {R2['customer_id'].iloc[0]}) with revenue of: {R2['total_revenue'].iloc[0].round(2)} ' )
            print(f'The Biggest store : {R3.index[0]} with revenue of: {R3['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest day : {R4.index[0].date()} with revenue of: {R4['total_revenue'].iloc[0].round(2)}')
    
            Final_df=Required.loc[:,['order_id','order_date','customer_id','first_name','last_name','revenue']]
            display(Final_df)
            
        else:
            print('Product id not present in database, please try again ')
            
    except Exception:
        print("Invalid input. Make sure input are in correct numerical (integer) format.")
        
###################################################################################################################################################
# Brand Analysis

def brand_analysis():
    def merge_all():
        Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                         .merge(customers, how='inner', on='customer_id')
                         .merge(stores, how='inner', on='store_id')
                         .merge(products, how='inner', on='product_id')
                         .merge(brands, how='inner', on='brand_id')
                         .merge(categories,how='inner',on='category_id'))
    
        Result=Merge_all
        return Result
    
    # creating merged dataframe
    Final=merge_all()
    
    try:
        brand_id_input=int(input('Please type the required brand id here (1-9): '))
        print('')
        unique_ids=Final['brand_id'].unique()
        
        if brand_id_input in unique_ids:
            Required=Final[Final['brand_id']==brand_id_input]
    
            # Product informaton
            brand_name=Required['brand_name'].iloc[0]
            brand_category=Required['category_name'].iloc[0]
    
            # Required answers-1 (Total products, Revenue, Total orders, AOV) (set of 4 major metrics)
            Total_products=Required['product_id'].nunique()
            Revenue=Required['revenue'].sum().round(2)
            Total_orders=Required['order_id'].nunique()
            AOV=(Revenue/Total_orders).round(2)
    
            # Required answers 2,3,4
            R2=Required.groupby(['product_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R3=Required.groupby(['customer_id','first_name','last_name'],as_index=False)\
                       .agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R4=Required.groupby(['store_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R5=Required.groupby(['order_date']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
    
            # Printing required answer
            print('Brand id: ',brand_id_input)
            print('Brand name: ',brand_name)
            print('Brand category: ',brand_category)
            print('')
            print('Total products under the brand: ',Total_products)
            print('Revenue generated: ',Revenue)
            print('Total orders: ',Total_orders)    
            print('Average Order Value: ',AOV)
            print('')
            print(f'The Biggest product : {R2.index[0]} with revenue of: {R2['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest customer : {R3['first_name'].iloc[0]} {R3['last_name'].iloc[0]} (customer id: {R3['customer_id'].iloc[0]}) with revenue of: {R3['total_revenue'].iloc[0].round(2)} ' )
            print(f'The Biggest store : {R4.index[0]} with revenue of: {R4['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest day : {R5.index[0].date()} with revenue of: {R5['total_revenue'].iloc[0].round(2)}')
    
            Final_df=Required.loc[:,['order_id','order_date','product_name','customer_id','first_name','last_name','revenue']]
            display(Final_df)
            
        else:
            print('Brand id not present in database, please try again ')
            
    except Exception:
        print("Invalid input. Make sure input are in correct numerical (integer) format.")

###################################################################################################################################################
# Category Analysis

def category_analysis():
    def merge_all():
        Merge_all=(orders.merge(order_items, how='inner', on='order_id')
                         .merge(customers, how='inner', on='customer_id')
                         .merge(stores, how='inner', on='store_id')
                         .merge(products, how='inner', on='product_id')
                         .merge(brands, how='inner', on='brand_id')
                         .merge(categories,how='inner',on='category_id'))
    
        Result=Merge_all
        return Result
    
    # creating merged dataframe
    Final=merge_all()
    
    try:
        category_id_input=int(input('Please type the required category id here (1-7): '))
        print('')
        unique_ids=Final['category_id'].unique()
        
        if category_id_input in unique_ids:
            Required=Final[Final['category_id']==category_id_input]
    
            # Product informaton
            category_name=Required['category_name'].iloc[0]
    
            # Required answers-1 (Total products, Revenue, Total orders, AOV) (set of 4 major metrics)
            Total_products=Required['product_id'].nunique()
            Total_brands=Required['brand_id'].nunique()
            Revenue=Required['revenue'].sum().round(2)
            Total_orders=Required['order_id'].nunique()
            AOV=(Revenue/Total_orders).round(2)
    
            # Required answers 2,3,4,5
            R2=Required.groupby(['product_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R3=Required.groupby(['brand_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R4=Required.groupby(['customer_id','first_name','last_name'],as_index=False)\
                       .agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R5=Required.groupby(['store_name']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
            R6=Required.groupby(['order_date']).agg(total_revenue=('revenue','sum')).sort_values('total_revenue',ascending=False).head(1)
    
            # Printing required answer
            print('Category id: ',category_id_input)
            print('Category name: ',category_name)
            print('')
            print('Total products under the category: ',Total_products)
            print('Total brands under the category: ',Total_brands)
            print('Revenue generated: ',Revenue)
            print('Total orders: ',Total_orders)    
            print('Average Order Value: ',AOV)
            print('')
            print(f'The Biggest product : {R2.index[0]} with revenue of: {R2['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest brand : {R3.index[0]} with revenue of: {R3['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest customer : {R4['first_name'].iloc[0]} {R4['last_name'].iloc[0]} (customer id: {R4['customer_id'].iloc[0]}) with revenue of: {R4['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest store : {R5.index[0]} with revenue of: {R5['total_revenue'].iloc[0].round(2)}')
            print(f'The Biggest day : {R6.index[0].date()} with revenue of: {R6['total_revenue'].iloc[0].round(2)}')
    
            Final_df=Required.loc[:,['order_id','order_date','product_name','brand_name','customer_id','first_name','last_name','revenue']]
            display(Final_df)
            
        else:
            print('Brand id not present in database, please try again ')
            
    except Exception:
        print("Invalid input. Make sure input are in correct numerical (integer) format.")

###################################################################################################################################################
# SELECTION
print('Select mode of analytics: \n 1) Product Analytics \n 2) Brand Analytics \n 3) Category Analytics')
print('')
Select=int(input('Please choose between 1 to 3: '))

if Select==1:
    product_analysis()
elif Select==2:
    brand_analysis()
else:
    category_analysis()

        
    

Select mode of analytics: 
 1) Product Analytics 
 2) Brand Analytics 
 3) Category Analytics



Please choose between 1 to 3:  2
Please type the required brand id here (1-9):  5



Brand id:  5
Brand name:  Ritchey
Brand category:  Mountain Bikes

Total products under the brand:  1
Revenue generated:  88486.02
Total orders:  77
Average Order Value:  1149.17

The Biggest product : Ritchey Timberwolf Frameset - 2016 with revenue of: 88486.02
The Biggest customer : Monika Berg (customer id: 46) with revenue of: 1499.88 
The Biggest store : Baldwin Bikes with revenue of: 59240.17
The Biggest day : 2016-03-20 with revenue of: 2999.72


,order_id,order_date,product_name,customer_id,first_name,last_name,revenue
9,4,2016-01-03,Ritchey Timberwolf Frameset - 2016,175,Nova,Hess,1499.78
47,18,2016-01-14,Ritchey Timberwolf Frameset - 2016,541,Lanita,Burton,1499.58
72,26,2016-01-18,Ritchey Timberwolf Frameset - 2016,1280,Jackeline,Colon,749.79
168,59,2016-02-05,Ritchey Timberwolf Frameset - 2016,563,Rodney,Odom,749.89
188,66,2016-02-09,Ritchey Timberwolf Frameset - 2016,1253,Susannah,Fields,749.79
...,...,...,...,...,...,...,...
3245,1124,2017-09-13,Ritchey Timberwolf Frameset - 2016,275,Pinkie,Kirkland,1499.88
3755,1296,2017-12-14,Ritchey Timberwolf Frameset - 2016,231,Jenee,Rasmussen,1499.78
3920,1345,2018-01-14,Ritchey Timberwolf Frameset - 2016,1037,Ashanti,Hammond,1499.58
4012,1378,2018-02-02,Ritchey Timberwolf Frameset - 2016,983,Marguerite,Berger,1499.58
